In [1]:
import numpy as np
import pandas as pd
from utils import *
import random
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from scipy import stats

from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

import h5py

/Applications/conda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Read smiles from data

In [2]:
smiles_pd = pd.read_csv('finalsmiles_pd.csv')
smiles_pd.head()

,smiles,lens
0,S([C@@H](C(=O)N1CCCC1)C)C1=NC(=O)c2c(N1)ccc(F)c2,48
1,O=C(NCC1(Cn2nccc2)CC1)c1nnc([O-])c2c1cccc2,42
2,S(=O)(=O)(CCN[C@@H]([C@H](C)n1nccc1)C)c1ccccc1,46
3,S(CCC(=O)N1CCNCC1)Cc1ccc(C)cc1,30
4,Fc1c(CNC2CCN(C(=O)c3cocc3)CC2)c(F)ccc1,38


In [3]:
smileslist = smiles_pd['smiles']

## Smiles to Char list

In [4]:
datawithnextline = '\n'.join(smileslist)

In [5]:
#chars = set(data)
def precess_smiles_as_charlist(data):

    allchars = []

    i = 0

    while i < len(data):

        if data[i] == 'B' and data[i+1] == 'r':
            allchars.append('Br')
            i += 1

        elif data[i] == 'C' and data[i+1] == 'l':
            allchars.append('Cl')
            i += 1

        else:    
            allchars.append(data[i])

        i += 1
    
    return allchars

## smiles to char list with '\n'

In [6]:
allcharswithn = precess_smiles_as_charlist(datawithnextline)
chars = set(allcharswithn)

### Dicts for char_to_index and index_to_char

* Atom: C H O N S P F Cl Br I B c o n s p
* bond: . = # : $
* electonric: + -
* @ \ /
* () []
* 1 2 3 4 5 6 7 8 9 %
* Other

## Note: '\n' is required in trainning procedure, not used for testing

In [7]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)
#print(char_to_ix)

{0: '\n', 1: '#', 2: '(', 3: ')', 4: '+', 5: '-', 6: '/', 7: '0', 8: '1', 9: '2', 10: '3', 11: '4', 12: '5', 13: '6', 14: '=', 15: '@', 16: 'Br', 17: 'C', 18: 'Cl', 19: 'F', 20: 'H', 21: 'I', 22: 'N', 23: 'O', 24: 'P', 25: 'S', 26: '[', 27: '\\', 28: ']', 29: 'c', 30: 'n', 31: 'o', 32: 'p', 33: 's'}


In [8]:
n_values = len(char_to_ix)

We dont have elements: 
.  : $ 7 8 9 Other , where '\\\' is '\'

## Use char_to_ix to indexing

In [9]:
# max len of smiles
maxLen = 60

In [ ]:
allsmilesindexlist = []
smileindexlist = []

for char in allcharswithn:
    
    if char != '\n':
        smileindexlist.append(char_to_ix[char])
    else:
        allsmilesindexlist.append(smileindexlist)
        smileindexlist = []

In [ ]:
len(allsmilesindexlist)

## Prepare X and Y 

In [ ]:
X = []
Y = []

for index in range(len(allsmilesindexlist)):
        
    sub_X = allsmilesindexlist[index]
    sub_Y = sub_X[1:] + [char_to_ix["\n"]]
        
    X.append(sub_X)
    Y.append(sub_Y)

## "-1" Padding Post

In [ ]:
X = pad_sequences(X, maxlen=maxLen, padding='post', value=-1)
Y = pad_sequences(Y, maxlen=maxLen, padding='post', value=-1)

In [ ]:
X[0]

In [ ]:
X.shape

## Save Padding Indexed Data

In [ ]:
h5f = h5py.File('PaddedIndexedData.h5', 'w')
h5f.create_dataset('indexed_padded_X', data=X)
h5f.create_dataset('indexed_padded_Y', data=Y)
h5f.close()

## Load Padding Indexed Data

In [ ]:
h5f = h5py.File('PaddedIndexedData.h5','r')
X = h5f['indexed_padded_X'][:]
Y = h5f['indexed_padded_Y'][:]
print(X.shape, Y.shape)
h5f.close()

In [ ]:
def one_hot(X, n_x):

    m = len(X)
    
    x = np.zeros((m, n_x), dtype=np.bool)
    
    for i, char_x in enumerate(X):    
        if char_x != -1:
            x[i, char_x] = 1
                
    return x

In [ ]:
X[0]

In [ ]:
Y[0]

In [ ]:
One_X = one_hot(X[0], n_values)
One_X.shape

In [ ]:
One_X[0]

In [ ]:
One_X[-1]

In [ ]:
FinalShape = X.shape + (n_values,)
FinalShape

## OneHot with Padding (First 100000)

In [ ]:
X_onehot = np.zeros((100000, 60, 34))

for i in range(100000):
     X_onehot[i, :, :] = one_hot(X[i, :], n_values)

In [ ]:
Y_onehot = np.zeros((100000, 60, 34))

for i in range(100000):
     Y_onehot[i, :, :] = one_hot(Y[i, :], n_values)

In [ ]:
print(X_onehot.shape)
print(Y_onehot.shape)

## Save OneHot Padding (first 100000)

In [ ]:
h5f = h5py.File('molecular_onehotPaddedIndexedData.h5', 'w')
h5f.create_dataset('onehot_indexed_padded_X', data=X_onehot)
h5f.create_dataset('onehot_indexed_padded_Y', data=Y_onehot)
h5f.close()

## Load OneHot Padding

In [10]:
h5f = h5py.File('molecular_onehotPaddedIndexedData.h5','r')
X_onehot = h5f['onehot_indexed_padded_X'][:]
Y_onehot = h5f['onehot_indexed_padded_X'][:]
print(X_onehot.shape, Y_onehot.shape)
h5f.close()

(100000, 60, 34) (100000, 60, 34)


## Mole2Vec_LTSM (set n_a for length of embedding)

In [11]:
#hidden state
n_a = 200

In [12]:
reshapor = Reshape((1, n_values))           # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

In [13]:
def Mole2Vec_LTSM(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the smiles
    n_a -- the number of activations used in our model
    n_values -- number of unique atom in smiles
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    # Step 1: Create empty list to append the outputs while you iterate 
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
    
    #outputs.append(a)
    
    # Step 3: Create model instance
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [14]:
print(maxLen)
print(n_a)
print(n_values)

60
200
34


In [15]:
model = Mole2Vec_LTSM(Tx = maxLen , n_a = n_a, n_values = n_values)

In [16]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
print(X_onehot.shape, Y_onehot.shape)

(100000, 60, 34) (100000, 60, 34)


In [18]:
Y_onehot = Y_onehot.reshape(Y_onehot.shape[1], Y_onehot.shape[0], Y_onehot.shape[2])

In [19]:
print(X_onehot.shape, Y_onehot.shape)

(100000, 60, 34) (60, 100000, 34)


In [20]:
len(X_onehot)

100000

In [21]:
batch_size = 100
a0 = np.zeros((batch_size, n_a))
finala = np.zeros((batch_size, n_a))
c0 = np.zeros((batch_size, n_a))
print(a0.shape)

(100, 200)


In [22]:
Y_onehot.shape

(60, 100000, 34)

In [23]:
loss = model.train_on_batch([X_onehot[0:0+batch_size, :, :], a0, c0], list(Y_onehot[:, 0:0+batch_size, :]))
loss[0]

146.48161

### Trainning Procedures (Only One Epoch)

In [24]:
batch_size = 100
a0 = np.zeros((batch_size, n_a))
c0 = np.zeros((batch_size, n_a))

for i in range(0, len(X_onehot), batch_size):
    loss = model.train_on_batch([X_onehot[i:i+batch_size, :, :], a0, c0], list(Y_onehot[:, i:i+batch_size, :]))
    if (i % 10000 == 0):   
        print("batch:", i, loss[0])

batch: 0 140.27757
batch: 10000 106.8993
batch: 20000 102.83084
batch: 30000 104.29698
batch: 40000 107.69733
batch: 50000 109.27319
batch: 60000 107.42186
batch: 70000 108.90444
batch: 80000 110.81684
batch: 90000 105.430115


In [25]:
a0 = np.zeros((1, n_a))
c0 = np.zeros((1, n_a))
model.predict([X_onehot[0:1, :, :], a0, c0])

[array([[1.5041321e-04, 1.0498307e-03, 1.0346197e-01, 9.2954457e-02,
         8.3543250e-04, 8.4982347e-03, 3.0392362e-03, 3.3850461e-04,
         7.2884008e-02, 4.6426315e-02, 1.0649579e-02, 9.8931813e-04,
         2.3332941e-04, 1.3482542e-04, 3.2104969e-02, 3.1331956e-02,
         7.9580006e-04, 1.9404598e-01, 1.9963463e-03, 5.4687187e-03,
         2.1192793e-02, 2.3205896e-04, 4.0883161e-02, 4.3841615e-02,
         2.6149253e-04, 3.9549163e-03, 3.0205613e-02, 1.1924367e-03,
         3.1736676e-02, 1.7951018e-01, 3.5020150e-02, 1.7863639e-03,
         1.8045490e-04, 2.6128527e-03]], dtype=float32),
 array([[7.44449153e-06, 1.13746535e-03, 9.84828547e-02, 9.98111963e-02,
         7.34369562e-04, 8.79194960e-03, 3.11048026e-03, 1.09766945e-04,
         6.86650202e-02, 4.56117541e-02, 1.13996407e-02, 1.13823137e-03,
         2.29256184e-05, 6.81510983e-06, 3.23534496e-02, 3.37844379e-02,
         6.52946008e-04, 1.79888651e-01, 2.68231588e-03, 5.22764167e-03,
         2.38872040e-02, 2

## Molecular to Vector

In [26]:
def Mole2Vec(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the smiles
    n_a -- the number of activations used in our model
    n_values -- number of unique atom in smiles
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
  
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
    
    
    # Step 3: finall embeddding a output
    model = Model(inputs=[X, a0, c0], outputs=a)
    
    ### END CODE HERE ###
    
    return model

## Test Molecular Embedding

In [31]:
model = Mole2Vec(Tx = maxLen , n_a = n_a, n_values = n_values)

In [32]:
a0 = np.zeros((1, n_a))
c0 = np.zeros((1, n_a))
smilesembedding = model.predict([X_onehot[0:1, :, :], a0, c0])
print(smilesembedding.shape)
smilesembedding

(1, 200)


array([[ 1.70674324e-01,  3.78974319e-01, -0.00000000e+00,
         4.96301660e-03, -3.02081883e-01, -3.29183817e-01,
        -0.00000000e+00,  1.41109094e-01,  3.86736810e-01,
        -2.11374789e-01,  0.00000000e+00, -2.09992558e-01,
         0.00000000e+00, -5.47144515e-03, -2.71852493e-01,
         2.12911755e-01,  6.26392569e-03, -3.88889611e-02,
         1.31163429e-02, -3.36230308e-01,  2.54168302e-01,
         3.06907654e-01,  1.48576930e-01,  2.17359099e-11,
         2.47958452e-01, -8.69196653e-02,  0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00, -8.46613199e-02,
        -3.20550323e-01,  3.25022101e-01,  3.16932857e-01,
         4.17709827e-01, -0.00000000e+00, -1.04115762e-01,
         3.39985490e-02,  1.78317130e-01,  2.78045045e-04,
        -3.54396887e-02, -2.79378831e-01,  0.00000000e+00,
        -1.18074395e-01,  1.55286696e-02,  3.73035669e-01,
        -2.15533980e-11, -5.39544132e-03,  2.55992770e-01,
        -2.65370816e-01, -1.50797039e-03,  2.20169291e-0